<a href="https://colab.research.google.com/github/nargesalavi/Quantum-Open-Source-Foundation-Mentorship/blob/master/QNN_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.1.0

In [ ]:
!pip install tensorflow-quantum

In [2]:
import tensorflow as tf
import tensorflow_quantum as tfq
import cirq
import sympy
import numpy as np

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
qubits = cirq.GridQubit.rect(1, 4)

In [4]:
def generate_even_block(block_number):
    """ Function for generating the even blocks
        block_number: Block number, it has to be even.
        return: ciq.Circuit """
    params = sympy.symbols(['theta_{}'.format(n) for n in range((block_number-1)*4,block_number*4)])

    # create the parameterized circuit
    circuit = cirq.Circuit(
        cirq.rz(params[0])(qubits[0]),
        cirq.rz(params[1])(qubits[1]),
        cirq.rz(params[2])(qubits[2]),
        cirq.rz(params[3])(qubits[3]),
        cirq.CZ(qubits[0],qubits[1]),
        cirq.CZ(qubits[0],qubits[2]),
        cirq.CZ(qubits[0],qubits[3]),
        cirq.CZ(qubits[1],qubits[2]),
        cirq.CZ(qubits[1],qubits[3]),
        cirq.CZ(qubits[2],qubits[3])
    )
    
    return circuit

In [5]:
def generate_odd_block(block_number):
    """ Function for generating the odd blocks
        block_number: Block number, it has to be odd.
        return: ciq.Circuit """
    params = sympy.symbols(['theta_{}'.format(n) for n in range((block_number-1)*4,block_number*4)])

    # create the parameterized circuit
    circuit = cirq.Circuit(
        cirq.rx(params[0])(qubits[0]),
        cirq.rx(params[1])(qubits[1]),
        cirq.rx(params[2])(qubits[2]),
        cirq.rx(params[3])(qubits[3])
    )
    
    return circuit

In [6]:
def generate_qnn(l):
    """ Function for generating qnn, containing l number of layers.
        l: number of layers, each layer contains one odd and one even block.
        return: ciq.Circuit """
    circuit = cirq.Circuit()
    for i in range(1,2*l+1):
        if i % 2 == 1:
            circuit += generate_odd_block(i)
        else:
            circuit += generate_even_block(i)
        
    return circuit

In [24]:
l=1
qnn = generate_qnn(l)
print(qnn)

                                               ┌──┐
(0, 0): ───Rx(theta_0)───Rz(theta_4)───@───@────@─────────────
                                       │   │    │
(0, 1): ───Rx(theta_1)───Rz(theta_5)───@───┼────┼@────@───────
                                           │    ││    │
(0, 2): ───Rx(theta_2)───Rz(theta_6)───────@────┼@────┼───@───
                                                │     │   │
(0, 3): ───Rx(theta_3)───Rz(theta_7)────────────@─────@───@───
                                               └──┘


In [8]:
params = tf.zeros([1,8*l ], tf.float32)
print(params)
params_names = sympy.symbols(['theta_{}'.format(n) for n in range(8*l)])

tf.Tensor([[0. 0. 0. 0. 0. 0. 0. 0.]], shape=(1, 8), dtype=float32)


In [9]:
state_layer = tfq.layers.State()
state = state_layer(c, symbol_names=params_names, symbol_values=params)
print(state)

Instructions for updating:
reduction_indices is deprecated, use axis instead
<tf.RaggedTensor [[(1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j]]>


In [10]:
random_circuit = cirq.testing.random_circuit(qubits = qubits,n_moments = np.random.randint(low=1,high=5),\
                                                                op_density = 0.99999999)
print(random_circuit)
random_circuit = tfq.layers.State()(random_circuit)
print(random_circuit)

(0, 0): ───iSwap───Y───
           │
(0, 1): ───iSwap───────

(0, 2): ───@───────Z───
           │
(0, 3): ───@───────Y───
<tf.RaggedTensor [[(-1.910685465164705e-15+0j), (4.371138828673793e-08+1.910685465164705e-15j), 0j, 0j, 0j, 0j, 0j, 0j, (4.371138828673793e-08+1.910685465164705e-15j), (-1-8.742277657347586e-08j), 0j, 0j, 0j, 0j, 0j, 0j]]>


In [11]:
diff = state[0] - random_circuit[0]
print(diff)

tf.Tensor(
[ 1.000000e+00+0.0000000e+00j -4.371139e-08-1.9106855e-15j
  0.000000e+00+0.0000000e+00j  0.000000e+00+0.0000000e+00j
  0.000000e+00+0.0000000e+00j  0.000000e+00+0.0000000e+00j
  0.000000e+00+0.0000000e+00j  0.000000e+00+0.0000000e+00j
 -4.371139e-08-1.9106855e-15j  1.000000e+00+8.7422777e-08j
  0.000000e+00+0.0000000e+00j  0.000000e+00+0.0000000e+00j
  0.000000e+00+0.0000000e+00j  0.000000e+00+0.0000000e+00j
  0.000000e+00+0.0000000e+00j  0.000000e+00+0.0000000e+00j], shape=(16,), dtype=complex64)


In [16]:
a = np.matrix(diff)
print(a*a.getH())

[[2.+0.j]]


In [50]:
def states_distance(state,target):
  diff = state[0] - target[0]
  diff = tf.reshape(diff, [16, 1])
  conjugate_transposed_state = tf.transpose(diff,conjugate=True)
  print(conjugate_transposed_state)
  return diff

In [51]:
a = states_distance(state,random_circuit)
print(a)

tf.Tensor(
[[ 1.000000e+00-0.0000000e+00j -4.371139e-08+1.9106855e-15j
   0.000000e+00-0.0000000e+00j  0.000000e+00-0.0000000e+00j
   0.000000e+00-0.0000000e+00j  0.000000e+00-0.0000000e+00j
   0.000000e+00-0.0000000e+00j  0.000000e+00-0.0000000e+00j
  -4.371139e-08+1.9106855e-15j  1.000000e+00-8.7422777e-08j
   0.000000e+00-0.0000000e+00j  0.000000e+00-0.0000000e+00j
   0.000000e+00-0.0000000e+00j  0.000000e+00-0.0000000e+00j
   0.000000e+00-0.0000000e+00j  0.000000e+00-0.0000000e+00j]], shape=(1, 16), dtype=complex64)
tf.Tensor(
[[ 1.000000e+00+0.0000000e+00j]
 [-4.371139e-08-1.9106855e-15j]
 [ 0.000000e+00+0.0000000e+00j]
 [ 0.000000e+00+0.0000000e+00j]
 [ 0.000000e+00+0.0000000e+00j]
 [ 0.000000e+00+0.0000000e+00j]
 [ 0.000000e+00+0.0000000e+00j]
 [ 0.000000e+00+0.0000000e+00j]
 [-4.371139e-08-1.9106855e-15j]
 [ 1.000000e+00+8.7422777e-08j]
 [ 0.000000e+00+0.0000000e+00j]
 [ 0.000000e+00+0.0000000e+00j]
 [ 0.000000e+00+0.0000000e+00j]
 [ 0.000000e+00+0.0000000e+00j]
 [ 0.000000e+00

In [28]:
adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)

In [29]:
distances = []
for i in range(1000):
  with tf.GradientTape() as tape:
    tape.watch(params)
    state = state_layer(qnn, symbol_names=params_names, symbol_values=params)
    distance = states_distance(state,random_circuit)
  distances.append(distance)
  gradients = tape.gradient(distance,params)
  adam_optimizer.apply_gradients(zip(gradients, params))



TypeError: ignored

In [53]:
x = np.arange(4).reshape(1, 4)
print(x)


[[0 1 2 3]]


In [54]:
y = np.arange(4).reshape(4, 1)
print(y)


[[0]
 [1]
 [2]
 [3]]


In [57]:
tf.keras.layers.Multiply()([x, y])

ValueError: ignored